# Serving reinforcement learning policy models
In this example, we train a reinforcement learning model and serve it
using Ray Serve.

We then instantiate an environment and step through it by querying the served model
for actions via HTTP.

Let's start with installing our dependencies:

In [1]:
!pip install -qU "ray[rllib,serve]" gym

In [1]:
!conda install tabulate -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /Users/cindyz/mambaforge/envs/fresh2

  added / updated specs:
    - tabulate


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    tabulate-0.8.10            |     pyhd8ed1ab_0          29 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          29 KB

The following NEW packages will be INSTALLED:

  tabulate           conda-forge/noarch::tabulate-0.8.10-pyhd8ed1ab_0



tabulate-0.8.10      | 29 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


Now we can run some imports:

In [ ]:
import gym
import numpy as np
import requests

from ray.air.checkpoint import Checkpoint
from ray.air.config import RunConfig
from ray.train.rl.rl_trainer import RLTrainer
from ray.air.config import ScalingConfig
from ray.train.rl.rl_predictor import RLPredictor
from ray.air.result import Result
from ray.serve import PredictorDeployment
from ray import serve
from ray.tune.tuner import Tuner

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

/Users/cindyz/mambaforge/envs/fresh2/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py:22: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


Since we'll be serving a reinforcement learning policy, we need to train one first. Thus we define a simple training function which will kick off online reinforcement learning of a PPO agent on the `CartPole-v0` environment.

In [3]:
def train_rl_ppo_online(num_workers: int, use_gpu: bool = False) -> Result:
    print("Starting online training")
    trainer = RLTrainer(
        run_config=RunConfig(stop={"training_iteration": 5}),
        scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu),
        algorithm="PPO",
        config={
            "env": "CartPole-v0",
            "framework": "tf",
        },
    )
    # Todo (krfricke/xwjiang): Enable checkpoint config in RunConfig
    # result = trainer.fit()
    tuner = Tuner(
        trainer,
        _tuner_kwargs={"checkpoint_at_end": True},
    )
    result = tuner.fit()[0]
    return result

Once we obtained a trained checkpoint, we will want to serve it using Ray Serve:

In [4]:
from ray.serve.drivers import DAGDriver
from ray.serve.http_adapters import json_request

def serve_rl_model(checkpoint: Checkpoint, name="RLModel") -> str:
    """Serve a RL model and return deployment URI.

    This function will start Ray Serve and deploy a model wrapper
    that loads the RL checkpoint into a RLPredictor.
    """
    deployment = PredictorDeployment.options(route_prefix=f"/RLModel")
    app = deployment.bind(RLPredictor, checkpoint)
    serve.run(app)
    return deployment.url

And to make sure everything works well, we can kick off an evaluation run on a fresh environment. This will query the served policy model to obtain actions using HTTP.

In [5]:
def evaluate_served_policy(endpoint_url: str, num_episodes: int = 3) -> list:
    """Evaluate a served RL policy on a local environment.

    This function will create an RL environment and step through it.
    To obtain the actions, it will query the deployed RL model.
    """
    env = gym.make("CartPole-v0")

    rewards = []
    for i in range(num_episodes):
        obs = env.reset()
        reward = 0.0
        done = False
        while not done:
            action = query_action(endpoint_url, obs)
            obs, r, done, _ = env.step(action)
            reward += r
        rewards.append(reward)

    return rewards


def query_action(endpoint_url: str, obs: np.ndarray):
    """Perform inference on a served RL model.

    This will send a HTTP request to the Ray Serve endpoint of the served
    RL policy model and return the result.
    """
    action_dict = requests.post(endpoint_url, json={"array": obs.tolist()}).json()
    return action_dict

Let's put it all together. First, we train the model:

In [6]:
num_workers = 2
use_gpu = False

result = train_rl_ppo_online(num_workers=num_workers, use_gpu=use_gpu)

Starting online training


2022-08-01 18:27:59,074	INFO worker.py:1481 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265.


Trial name,status,loc,iter,total time (s),ts,reward,num_recreated_wor...,episode_reward_max,episode_reward_min
AIRPPO_581a9_00000,TERMINATED,127.0.0.1:97553,5,80.6556,20000,122.57,0,200,17


/Users/cindyz/ray/python/ray/util/placement_group.py:78: DeprecationWarning: placement_group parameter is deprecated. Use scheduling_strategy=PlacementGroupSchedulingStrategy(...) instead, see the usage at https://docs.ray.io/en/master/ray-core/package-ref.html#ray-remote.
  return bundle_reservation_check.options(
/Users/cindyz/ray/python/ray/_private/ray_option_utils.py:266: DeprecationWarning: Setting 'object_store_memory' for actors is deprecated since it doesn't actually reserve the required object store memory. Use object spilling that's enabled by default (https://docs.ray.io/en/master/ray-core/objects/object-spilling.html) instead to bypass the object store memory size limitation.
  warnings.warn(
/Users/cindyz/ray/python/ray/actor.py:637: DeprecationWarning: placement_group parameter is deprecated. Use scheduling_strategy=PlacementGroupSchedulingStrategy(...) instead, see the usage at https://docs.ray.io/en/master/ray-core/package-ref.html#ray-remote.
  return actor_cls._remot

(RolloutWorker pid=97558) Metal device set to: Apple M1 Pro
(RolloutWorker pid=97559) Metal device set to: Apple M1 Pro
(AIRPPO pid=97553) Metal device set to: Apple M1 Pro


(AIRPPO pid=97553) 2022-08-01 18:28:05,005	WARNING util.py:65 -- Install gputil for GPU system monitoring.


Result for AIRPPO_581a9_00000:
  agent_timesteps_total: 4000
  counters:
    num_agent_steps_sampled: 4000
    num_agent_steps_trained: 4000
    num_env_steps_sampled: 4000
    num_env_steps_trained: 4000
  custom_metrics: {}
  date: 2022-08-01_18-28-22
  done: false
  episode_len_mean: 20.107142857142858
  episode_media: {}
  episode_reward_max: 86.0
  episode_reward_mean: 20.107142857142858
  episode_reward_min: 9.0
  episodes_this_iter: 196
  episodes_total: 196
  experiment_id: a9f0e1f6b8e749e68b4cfb3009a5b024
  hostname: Cindys-MacBook-Pro-16
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 4.999999873689376e-05
          entropy: 0.6651962995529175
          entropy_coeff: 0.0
          kl: 0.028715724125504494
          model: {}
          policy_loss: -0.03502190113067627
          total_loss: 8.867462158203125
          vf_explained_var: -0.07922156900167465
          vf_l

Result for AIRPPO_581a9_00000:
  agent_timesteps_total: 12000
  counters:
    num_agent_steps_sampled: 12000
    num_agent_steps_trained: 12000
    num_env_steps_sampled: 12000
    num_env_steps_trained: 12000
  custom_metrics: {}
  date: 2022-08-01_18-28-53
  done: false
  episode_len_mean: 63.93
  episode_media: {}
  episode_reward_max: 200.0
  episode_reward_mean: 63.93
  episode_reward_min: 10.0
  episodes_this_iter: 39
  episodes_total: 331
  experiment_id: a9f0e1f6b8e749e68b4cfb3009a5b024
  hostname: Cindys-MacBook-Pro-16
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.30000001192092896
          cur_lr: 4.999999873689376e-05
          entropy: 0.5743724703788757
          entropy_coeff: 0.0
          kl: 0.00936416257172823
          model: {}
          policy_loss: -0.01809571124613285
          total_loss: 9.57343578338623
          vf_explained_var: 0.00913438480347395
          vf_loss: 9.588722229003906


Result for AIRPPO_581a9_00000:
  agent_timesteps_total: 20000
  counters:
    num_agent_steps_sampled: 20000
    num_agent_steps_trained: 20000
    num_env_steps_sampled: 20000
    num_env_steps_trained: 20000
  custom_metrics: {}
  date: 2022-08-01_18-29-25
  done: true
  episode_len_mean: 122.57
  episode_media: {}
  episode_reward_max: 200.0
  episode_reward_mean: 122.57
  episode_reward_min: 17.0
  episodes_this_iter: 23
  episodes_total: 380
  experiment_id: a9f0e1f6b8e749e68b4cfb3009a5b024
  hostname: Cindys-MacBook-Pro-16
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.30000001192092896
          cur_lr: 4.999999873689376e-05
          entropy: 0.5369527339935303
          entropy_coeff: 0.0
          kl: 0.0034114511217921972
          model: {}
          policy_loss: -0.011496230028569698
          total_loss: 9.521854400634766
          vf_explained_var: 0.028535105288028717
          vf_loss: 9.5323276519

2022-08-01 18:29:26,574	INFO tune.py:758 -- Total run time: 87.01 seconds (86.19 seconds for the tuning loop).


Then, we serve it using Ray Serve:

In [7]:
endpoint_url = serve_rl_model(result.checkpoint)

(ServeController pid=97764) INFO 2022-08-01 18:29:27,052 controller 97764 http_state.py:123 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-54cba86a01db964a880dcfa006e732495e0c3161fbf3eaac0d0befc6' on node '54cba86a01db964a880dcfa006e732495e0c3161fbf3eaac0d0befc6' listening on '127.0.0.1:8000'
(ServeController pid=97764) INFO 2022-08-01 18:29:27,568 controller 97764 deployment_state.py:1232 - Adding 1 replicas to deployment 'PredictorDeployment'.
(HTTPProxyActor pid=97766) INFO:     Started server process [97766]
(ServeReplica:PredictorDeployment pid=97773) /Users/cindyz/mambaforge/envs/work/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py:22: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
(ServeReplica:PredictorDeployment pid=97773)   import imp
(ServeReplica:PredictorDeployment pid=97773) 2022-08-01 18:29:29,749	INFO algorithm.py:1871 -- Your framework setting is

(RolloutWorker pid=97778) Metal device set to: Apple M1 Pro
(RolloutWorker pid=97777) Metal device set to: Apple M1 Pro
(ServeReplica:PredictorDeployment pid=97773) Metal device set to: Apple M1 Pro


(ServeReplica:PredictorDeployment pid=97773) 2022-08-01 18:29:32,428	WARNING util.py:65 -- Install gputil for GPU system monitoring.
(ServeReplica:PredictorDeployment pid=97773) 2022-08-01 18:29:32,461	INFO trainable.py:668 -- Restored on 127.0.0.1 from checkpoint: /var/folders/cp/nv6dm3zs3dd91qxn49k6kd_m0000gn/T/checkpoint_tmp_ittjno4s
(ServeReplica:PredictorDeployment pid=97773) 2022-08-01 18:29:32,461	INFO trainable.py:677 -- Current state after restoring: {'_iteration': 5, '_timesteps_total': None, '_time_total': 80.65559387207031, '_episodes_total': 380}
You are retrieving a sync handle inside an asyncio loop. Try getting Deployment.get_handle(.., sync=False) to get better performance. Learn more at https://docs.ray.io/en/latest/serve/handle-guide.html#sync-and-async-handles
/Users/cindyz/ray/python/ray/serve/_private/router.py:75: DeprecationWarning: The loop argument is deprecated since Python 3.8, and scheduled for removal in Python 3.10.
  self.config_updated_event = asyncio.E

And then we evaluate the served model on a fresh environment:

In [8]:
rewards = evaluate_served_policy(endpoint_url=endpoint_url)

/Users/cindyz/mambaforge/envs/work/lib/python3.8/site-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1` with the environment ID `CartPole-v1`.
  logger.warn(
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:32,643 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 5.7ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:32,692 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 47.9ms
(ServeReplica:PredictorDeployment pid=97773) INFO 2022-08-01 18:29:32,641 PredictorDeployment PredictorDeployment#nFSuyR replica.py:482 - HANDLE __call__ OK 0.2ms
(ServeReplica:PredictorDeployment pid=97773) INFO 2022-08-01 18:29:32,690 PredictorDeployment PredictorDeployment#nFSuyR replica.py:482 - HANDLE __call__ OK 43.6ms
(ServeReplica:PredictorDeployment pid=97773) INFO 2022-08-01 18:29:32,697 PredictorDeployment PredictorDeployment#nFSuyR replica.py:482 - HANDLE __call__ OK 0.1ms
(HTTPProxyA

(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,010 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 8.7ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,015 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 3.8ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,025 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 9.1ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,030 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 3.7ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,040 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 8.9ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,046 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 3.7ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,056 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 9.0ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,061 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 3.6ms
(HTTPProxyActor pid=97766) INFO 

(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,215 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 9.3ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,221 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.0ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,231 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 9.1ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,236 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 3.8ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,248 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 10.1ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,254 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.2ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,264 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 9.6ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,269 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 3.8ms
(HTTPProxyActor pid=97766) INFO

(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,436 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 9.2ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,441 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 3.7ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,451 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 9.0ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,457 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 3.8ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,467 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 8.8ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,472 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.0ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,482 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 8.5ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,487 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 3.6ms
(HTTPProxyActor pid=97766) INFO 

(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,855 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 9.6ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,860 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 3.8ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,870 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 8.9ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,875 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 3.7ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,885 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 8.7ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,890 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 3.8ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,900 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 8.7ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:33,905 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.0ms
(HTTPProxyActor pid=97766) INFO 

...

(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:39,508 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 8.9ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:39,513 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 3.8ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:39,523 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 9.0ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:39,529 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.5ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:39,540 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 9.2ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:39,546 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.7ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:39,556 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 9.1ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:39,562 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.3ms
(HTTPProxyActor pid=97766) INFO 

(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:39,936 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 8.9ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:39,942 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.0ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:39,952 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 9.4ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:39,958 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.1ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:39,969 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 10.3ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:39,976 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.7ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:39,989 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 11.8ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:39,995 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.4ms
(HTTPProxyActor pid=97766) INF

(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:40,354 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 10.3ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:40,360 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.3ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:40,371 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 9.5ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:40,377 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.6ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:40,388 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 9.7ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:40,394 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 3.9ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:40,404 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 9.3ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:40,411 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.8ms
(HTTPProxyActor pid=97766) INFO

(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:40,774 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 11.2ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:40,780 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.5ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:40,793 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 11.6ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:40,799 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.1ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:40,809 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 9.3ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:40,815 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 3.8ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:40,824 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 8.8ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:40,830 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 3.9ms
(HTTPProxyActor pid=97766) INF

(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:41,195 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.9ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:41,208 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 11.6ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:41,213 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.0ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:41,225 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 10.5ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:41,231 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.1ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:41,242 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 10.1ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:41,248 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 307 4.3ms
(HTTPProxyActor pid=97766) INFO 2022-08-01 18:29:41,260 http_proxy 127.0.0.1 http_proxy.py:315 - POST /RLModel 200 11.2ms
(HTTPProxyActor pid=97766) I

In [10]:
print("Episode rewards:", rewards)

Episode rewards: [171.0, 200.0, 174.0]


After we're done, we can shutdown Ray Serve.

In [11]:
serve.shutdown()

(ServeController pid=97764) INFO 2022-08-01 18:30:05,897 controller 97764 deployment_state.py:1257 - Removing 1 replicas from deployment 'PredictorDeployment'.
(ServeReplica:PredictorDeployment pid=97773) /Users/cindyz/mambaforge/envs/work/lib/python3.8/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 3 leaked semaphore objects to clean up at shutdown
(ServeReplica:PredictorDeployment pid=97773)   warnings.warn('resource_tracker: There appear to be %d '
